<a href="https://colab.research.google.com/github/NakkaBhavith/AI/blob/main/AI_Assign5_450.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [33]:
import numpy as np
import math as mt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [34]:
data = pd.read_excel('/content/drive/MyDrive/AI Data/Assign5_data.xlsx')
data.head()

,Unnamed: 0,L(T-3),L(T-2),L(T-1),L(T)
0,1,5551.82208,4983.17184,4888.39680,5072.95872
1,2,5196.25980,5641.29720,7153.17732,7849.80504
2,3,6329.97504,6292.87560,5349.80160,5225.40936
3,4,5000.47452,4775.53968,6177.05676,6177.05676
4,5,6155.70120,6212.12976,7424.25264,7407.41760


In [ ]:
def normalize(column):
    c_max, c_min = np.max(column), np.min(column)
    for i in range(len(column)):
        column.iloc[i] = (column.iloc[i] - c_min) / (c_max - c_min)
    return c_max, c_min
x1max, x1min = normalize(data['L(T-3)'])
x2max, x2min = normalize(data['L(T-2)'])
x3max, x3min = normalize(data['L(T-1)'])
ymax, ymin = normalize(data['L(T)'])
data.head()

In [46]:
data.isnull().sum()

Unnamed: 0    0
L(T-3)        0
L(T-2)        0
L(T-1)        0
L(T)          0
dtype: int64

In [37]:
X = data [['L(T-3)', 'L(T-2)', 'L(T-1)']]
Y = data ['L(T)']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.1, random_state=101)

In [38]:
m = [np.random.uniform(-1, 1) for i in range(xtrain.shape[1])]
c = np.random.uniform(-1, 1)
eeta = 0.001
epochs = 10

In [39]:
for _ in range(epochs):
    for i in range(xtrain.shape[0]):
        y, ypred = ytrain.iloc[i], sum([xtrain.iat[i,z] * m[z] for z in range(xtrain.shape[1])]) + c
        gradm = [-1 * xtrain.iat[i,j] * (y - ypred) for j in range(xtrain.shape[1])]
        gradc = -1 * (y - ypred)
        c = c - eeta * gradc
        for j in range(len(m)):
            m[j] = m[j] - eeta * gradm[j]
print(m, c)

[-0.4679652901110725, 0.3085161562014834, 0.25876486243616204] 0.10876454698745668


In [40]:
ytrainPred = []
for x in range(xtrain.shape[0]):
    ytrainPred.append(sum([xtrain.iat[x, z] * m[z] for z in range(xtrain.shape[1])]) + c)
ytestPred = []
for x in range(xtest.shape[0]):
    ytestPred.append(sum([xtest.iat[x, z] * m[z] for z in range(xtest.shape[1])]) + c)

In [ ]:
xtrain['L(T-3)'] = xtrain['L(T-3)'] * (x1max - x1min) + x1min
xtrain['L(T-2)'] = xtrain['L(T-2)'] * (x2max - x2min) + x2min
xtrain['L(T-1)'] = xtrain['L(T-1)'] * (x3max - x3min) + x3min
ytrain = ytrain * (ymax - ymin) + ymin
xtest['L(T-3)'] = xtest['L(T-3)'] * (x1max - x1min) + x1min
xtest['L(T-2)'] = xtest['L(T-2)'] * (x2max - x2min) + x2min
xtest['L(T-1)'] = xtest['L(T-1)'] * (x3max - x3min) + x3min
ytest = ytest * (ymax - ymin) + ymin
ytrainPred = np.multiply(ytrainPred, (ymax - ymin)) + ymin
ytestPred = np.multiply(ytestPred, (ymax - ymin)) + ymin

In [42]:
#Compute the error for training and testing data
MAE_Train = mean_absolute_error(ytrainPred, ytrain) #Testing MAE
MSE_Train = mean_squared_error(ytrainPred, ytrain) #Testing MSE
RMSE_Train = mean_squared_error(ytrainPred, ytrain) ** 0.5 #Testing RMSE
MAE_Test = mean_absolute_error(ytestPred, ytest) #Testing MAE
MSE_Test = mean_squared_error(ytestPred, ytest) #Testing MSE
RMSE_Test = mean_squared_error(ytestPred, ytest) ** 0.5 #Testing RMSE
print('Training Error :')
print('Mean Absolute Error :', MAE_Train)
print('Mean Squared Error :', MSE_Train)
print('Root Mean Squared Error :', RMSE_Train)
print('\nTesting Error :')
print('Mean Absolute Error :', MAE_Test)
print('Mean Squared Error :', MSE_Test)
print('Root Mean Squared Error :', RMSE_Test)

Training Error :
Mean Absolute Error : 792.5684105920789
Mean Squared Error : 1016126.3448983339
Root Mean Squared Error : 1008.030924574407

Testing Error :
Mean Absolute Error : 341.4915997170683
Mean Squared Error : 116616.5126773224
Root Mean Squared Error : 341.4915997170683


In [ ]:
xtrain['Y_Train'] = ytrain
xtrain['Y_Train_Prediction'] = ytrainPred
xtrain

In [ ]:
xtest['Y_Test'] = ytest
xtest['Y_Test_Prediction'] = ytestPred
xtest

In [45]:
#Deployment
x1 = float(input('Enter load at T-3 hours : '))
x2 = float(input('Enter load at T-2 hours : '))
x3 = float(input('Enter load at T-1 hours : '))
x1Norm = (x1 - x1min) / (x1max - x1min)
x2Norm = (x2 - x2min) / (x2max - x2min)
x3Norm = (x3 - x3min) / (x3max - x3min)
yPred = x1Norm * m[0] + x2Norm * m[1] + x3Norm * m[2] + c
y = yPred * (ymax - ymin) + ymin
print('Load at T hour :', y)3

Enter load at T-3 hours : 1500
Enter load at T-2 hours : 2000
Enter load at T-1 hours : 2500
Load at T hour : 5034.538287431769
